<a href="https://colab.research.google.com/github/haosulab/SAPIEN-tutorial/blob/master/robotics/1_getting_started_with_robot.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Note: Some core features of SAPIEN are not available on Colab, including the interactive viewer and ray-tracing functionalities. You need to run SAPIEN locally for full features. You can also find the latest SAPIEN tutorial at [SAPIEN's documentation](https://sapien.ucsd.edu/docs/latest/index.html).

# Robotics in SAPIEN

In this tutorial series, you will learn how to use SAPIEN to work with robots and tackle basic robotics tasks.

# Robotics Tutorial 1: Getting Started with Robot

In this tutorial, you will learn the following:

- Load a robot (URDF)
- Set joint positions
- Compensate passive forces
- Control the robot by torques

## Preparation

> Note: you need GPU runtime to run the notebook.

In [ ]:
%pip install sapien

import sapien.core as sapien
import numpy as np
import matplotlib.pyplot as plt
import os

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

%matplotlib inline

In [ ]:
# Hyperparameter
balance_passive_force = False

## Set up the engine, renderer and scene

First of all, let's set up the simulation environment. You might have been very familiar with this process.

In [ ]:
engine = sapien.Engine()
engine.set_log_level("critical")
renderer = sapien.SapienRenderer()
engine.set_renderer(renderer)

scene_config = sapien.SceneConfig()
scene = engine.create_scene(scene_config)
scene.set_timestep(1 / 240.0)
scene.add_ground(0)

# Light and camera are for later visualization
scene.set_ambient_light([0.5, 0.5, 0.5])
scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5])
camera = scene.add_camera(name='camera', width=1024, height=768, fovy=1.57, near=0.1, far=100)
camera.set_pose(sapien.Pose([-0.9, 0, 0.6], [0.988771, 0, 0.149438, 0]))

## Load a robot URDF

Now, you can create a `URDFLoader` to load the URDF XML of Kinova Jaco2 arm. [URDF XML](http://wiki.ros.org/urdf/XML) describes a robot. Usually, URDF files are provided by manufacturers. For example, the URDF XML of Kinova Jaco2 arm can be found [here](https://github.com/Kinovarobotics/kinova-ros).

In [ ]:
if IN_COLAB:
    !gdown -q 10-ASSF6hiMLoMfajrTBMNn_U3YyP6-sl
    !unzip -o -q jaco2.zip
    assets_dir = "."
else:
    assets_dir = "../assets"

loader: sapien.URDFLoader = scene.create_urdf_loader()
print("fix_root_link: ", loader.fix_root_link)
robot: sapien.Articulation = loader.load(os.path.join(assets_dir, "jaco2/jaco2.urdf"))

Note that there is a `fix_root_link` flag for the URDF loader. If it is true (by default), then the root link of the robot will be fixed. Otherwise, it is allowed to move freely.

> Note: When a robot is already loaded, changing the flag of the URDF loader will not take effect.

The robot is loaded as `Articulation`, which is a tree of links connected by joints. We can set the pose of its root link through `set_root_pose(...)`.

In [ ]:
# Set root link pose
robot.set_root_pose(sapien.Pose([0, 0, 0], [1, 0, 0, 0]))

## Set joint positions

We can also set initial joint positions through `set_qpos(qpos=...)`. The `qpos` should be a concatenation of the position of each joint. Its length is the degree of freedom, and its order is the same as that returned by `robot.get_joints()`.

> Note: If the articulation is loaded from a URDF file, its joints are in preorder (DFS preorder traversal over the articulation tree). If the articulation is built programmatically (as introduced in *Basics Tutorial 4: Create Articulations*), its joints are in the order when they are built.

In [ ]:
# Set initial joint positions
arm_init_qpos = [4.71, 2.84, 0, 0.75, 4.62, 4.48, 4.88]
gripper_init_qpos = [0, 0, 0, 0, 0, 0]
init_qpos = arm_init_qpos + gripper_init_qpos
robot.set_qpos(init_qpos)

Let's see what will happen if we run the simulation loop. Run the following cell and you will see 10 screenshots of the robot during the simulation. If you are running locally, an additional viewer window will be shown to display the animation of the simulation.

In [ ]:
plt.figure(figsize=(15, 6))
print("robot initial qpos: ", robot.get_qpos())

if IN_COLAB:
    for steps in range(200):
        if balance_passive_force: # This will be explained in next section
            qf = robot.compute_passive_force(gravity=True, coriolis_and_centrifugal=True)
            robot.set_qf(qf)

        scene.step()

        if steps % 20 == 0:
            scene.update_render()
            camera.take_picture()
            rgba = camera.get_color_rgba()
            plt.subplot(2, 5, steps//20+1)
            plt.imshow(rgba[..., :3])
    
    print("robot qpos at steps 199: ", robot.get_qpos())
else:
    from sapien.utils import Viewer
    viewer = Viewer(renderer)
    viewer.set_scene(scene)
    viewer.set_camera_xyz(x=-0.8, y=0, z=0.4)
    viewer.set_camera_rpy(r=0, p=-0.3, y=0)

    steps = 0
    while not viewer.closed:
        if balance_passive_force: # This will be explained in next section
            qf = robot.compute_passive_force(gravity=True, coriolis_and_centrifugal=True)
            robot.set_qf(qf)

        scene.step()
        scene.update_render()
        viewer.render()

        if steps < 200 and steps % 20 == 0:
            scene.update_render()
            camera.take_picture()
            rgba = camera.get_color_rgba()
            plt.subplot(2, 5, steps//20+1)
            plt.imshow(rgba[..., :3])
        
        if steps == 199:
            print("robot qpos at steps 199: ", robot.get_qpos())

        steps += 1

You will see that the robot arm fell down (due to gravity and other possible forces, like coriolis and centrifugal force). Let's find out how to keep the robot at a certain pose.

## Compensate passive forces

For a real robot, gravity compensation is done by an internal controller hardware. So it is usually desirable to skip this troublesome calculation of how to compensate gravity. SAPIEN provides `compute_passive_force` to compute desired forces or torques on joints to compensate passive forces:
    
    qf = robot.compute_passive_force(gravity=True, coriolis_and_centrifugal=True)
    robot.set_qf(qf)

In this example, we only consider gravity as well as coriolis and centrifugal force.

Change the hyperparameter `balance_passive_force` at the top to `True` and rerun the notebook. You should observe that this time the robot can stay at the target pose. However, note that there is slight numeric differences between the initial `qpos` and the `qpos` at steps 200. In fact, due to numeric error, the robot will gradually deviate from the target pose. If you are running locally and visualizing with a viewer, you might even observe this deviation as the simulation continues.

> Note: To avoid deviating from the target pose gradually, either we specify the damping (resistence proportional to velocity) of each joint in the URDF XML, or a controller can be used to compute desired extra forces or torques to keep the robot around the target pose. In the next tutorial, we will elaborate how to control the robot with a controller.